In [ ]:
!pip install gdown

In [ ]:
!gdown 1NhHyFv0nsL0hsvKBQKwHEt4oNeKqLxcE

In [ ]:
!unzip Digit.zip

In [ ]:
pip install numpy pillow tensorflow

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from PIL import Image  # Import Pillow
import matplotlib.pyplot as plt

# Unzip dataset
!unzip digit.zip -d /content/

# Define dataset paths
train_dir = "/content/DevanagariHandwrittenDigitDataset/Test"
test_dir = "/content/DevanagariHandwrittenDigitDataset/Test"

# Define image size
img_height, img_width = 28, 28

# Function to load images and labels using PIL
def load_images_from_folder(folder):
    images = []
    labels = []
    class_names = sorted(os.listdir(folder))  # Sorted class names (digit_0, digit_1, ...)
    class_map = {name: i for i, name in enumerate(class_names)}  # Map class names to labels

    for class_name in class_names:
        class_path = os.path.join(folder, class_name)
        label = class_map[class_name]

        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)

            # Load image using PIL
            img = Image.open(img_path).convert("L")  # Convert to grayscale
            img = img.resize((img_width, img_height))  # Resize to (28,28)
            img = np.array(img) / 255.0  # Normalize pixel values to [0,1]

            images.append(img)
            labels.append(label)

    return np.array(images), np.array(labels), class_names

# Load training and testing datasets
x_train, y_train, class_names = load_images_from_folder(train_dir)
x_test, y_test, _ = load_images_from_folder(test_dir)

# Reshape images for Keras input (adding channel dimension)
x_train = x_train.reshape(-1, img_height, img_width, 1)  # Shape (num_samples, 28, 28, 1)
x_test = x_test.reshape(-1, img_height, img_width, 1)

# One-hot encode labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Print dataset shape
print(f"Training set: {x_train.shape}, Labels: {y_train.shape}")
print(f"Testing set: {x_test.shape}, Labels: {y_test.shape}")

# Visualize all digits
plt.figure(figsize=(10, 5))
for i, class_name in enumerate(class_names):
    img_index = np.where(y_train.argmax(axis=1) == i)[0][0]  # Get an index for each class
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_train[img_index].reshape(28, 28), cmap='gray')
    plt.title(f"Label: {class_name}")
    plt.axis("off")
plt.show()

4

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Apply Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)

datagen.fit(x_train)

# Build an Improved CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),  # Dropout to reduce overfitting
    Dense(10, activation='softmax')  # 10 classes
])

# Compile the Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks for Better Training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                    validation_data=(x_test, y_test),
                    epochs=50,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the Model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Plot Training History
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.tight_layout()
plt.show()

5

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

# Print the evaluation results
print(f"Test Accuracy: {test_acc * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

6

In [ ]:
# Save the trained model to an HDF5 file
model.save("handwritten_digit_model.h5")
print("Model saved successfully!")

In [ ]:
# Load the saved model
loaded_model = tf.keras.models.load_model("handwritten_digit_model.h5")
print("Model loaded successfully!")

In [ ]:
# Evaluate the loaded model on the test dataset
test_loss, test_acc = loaded_model.evaluate(x_test, y_test, verbose=2)

# Print the results
print(f"Loaded Model Test Accuracy: {test_acc * 100:.2f}%")
print(f"Loaded Model Test Loss: {test_loss:.4f}")

7

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Make predictions on the test set
predictions = model.predict(x_test)

# Convert probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Convert one-hot encoded y_test back to class labels for comparison
true_labels = np.argmax(y_test, axis=1)

# Select one sample for each digit (0-9)
num_samples = 10
selected_indices = []

# Loop to select one sample for each digit (0-9)
for digit in range(10):
    digit_indices = np.where(true_labels == digit)[0]
    selected_index = np.random.choice(digit_indices)
    selected_indices.append(selected_index)

# Display the selected samples
plt.figure(figsize=(12, 5))
for i, index in enumerate(selected_indices):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_test[index].reshape(28, 28), cmap='gray')
    plt.title(f"Pred: {predicted_labels[index]}\nTrue: {true_labels[index]}")
    plt.axis("off")

plt.tight_layout()
plt.show()